In [ ]:
!pip install gdal

In [ ]:
!apt-get install -y gdal-bin python-gdal python3-gdal
!pip install rasterio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package python-gdal is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gdal-bin

E: Package 'python-gdal' has no installation candidate
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 31.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/


'Colab Notebooks'   insat_data	 SIH1738_cog  'Untitled spreadsheet.gsheet'


In [ ]:
!mkdir -p /content/drive/MyDrive/insat_data


In [ ]:
!tar -xvf /content/drive/MyDrive/SIH1738_cog.tar -C /content/drive/MyDrive/insat_data/


tar: /content/drive/MyDrive/SIH1738_cog.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
!apt-get install gdal-bin



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-gdal python3-numpy
Suggested packages:
  libgdal-grass python-numpy-doc python3-pytest
The following NEW packages will be installed:
  gdal-bin python3-gdal python3-numpy
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,055 kB of archives.
After this operation, 25.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1.21.5-1ubuntu22.04.1 [3,467 kB]
Get:2 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 python3-gdal amd64 3.6.4+dfsg-1~jammy0 [1,027 kB]
Get:3 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 gdal-bin amd64 3.6.4+dfsg-1~jammy0 [561 kB]
Fetched 5,055 kB in 5s (985 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 123599 files and directo

In [ ]:
!pip install gdal


In [ ]:
!ls /content/drive/MyDrive/insat_data/


output.cog


In [ ]:
!gdal_translate /content/drive/MyDrive/insat_data/SIH2024.tif /content/drive/MyDrive/insat_data/output_cog.tif -of COG



ERROR 4: /content/drive/MyDrive/insat_data/SIH2024.tif: No such file or directory


In [ ]:
!mkdir -p /content/drive/MyDrive/insat_data/SIH2024

In [ ]:
!gdal_translate /content/drive/MyDrive/insat_data/SIH2024.tif /content/drive/MyDrive/insat_data/output_cog.tif -of COG



ERROR 4: /content/drive/MyDrive/insat_data/SIH2024.tif: No such file or directory


In [ ]:
!ls /content/drive/MyDrive/insat_data/


output.cog  SIH2024


In [ ]:
!gdal_translate /content/drive/MyDrive/SIH1738_cog/SIH2024 /content/drive/MyDrive/insat_data/SIH2024OUTPUT -of COG


ERROR 4: `/content/drive/MyDrive/SIH1738_cog/SIH2024' not recognized as a supported file format.


In [ ]:
import os

directory = '/content/drive/MyDrive/insat_data/SIH2024OUTPUT/'
files = [f for f in os.listdir(directory) if f.endswith('.h5')]
print(files)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/insat_data/SIH2024OUTPUT/'

In [ ]:
!pip install h5py
!pip install rasterio

In [ ]:
import h5py

def inspect_datasets(input_hdf5_file):
    with h5py.File(input_hdf5_file, 'r') as file:
        for dataset_name in file.keys():
            dataset = file[dataset_name]
            print(f'Dataset: {dataset_name}')
            print(f'Shape: {dataset.shape}')
            print(f'Dtype: {dataset.dtype}')
            print()

input_hdf5_file = '/content/drive/MyDrive/SIH1738_cog/SIH2024/3RIMG_04SEP2024_1115_L1C_ASIA_MER_V01R00.h5'
inspect_datasets(input_hdf5_file)


In [ ]:
import h5py

def list_hdf5_datasets(file_path):
    with h5py.File(file_path, 'r') as file:
        print(f"File: {file_path}")
        def print_dataset(name, obj):
            if isinstance(obj, h5py.Dataset):
                print(f"Dataset: {name}")
        file.visititems(print_dataset)

# List datasets in all HDF5 files in the directory
import os

directory = '/content/drive/MyDrive/insat_data/SIH2024OUTPUT/'
for file_name in os.listdir(directory):
    if file_name.endswith('.h5'):
        file_path = os.path.join(directory, file_name)
        list_hdf5_datasets(file_path)


In [ ]:
def process_hdf5_to_geotiff(input_hdf5_file, output_geotiff_file, dataset_name):
    try:
        with h5py.File(input_hdf5_file, 'r') as file:
            if dataset_name in file:
                dataset = file[dataset_name][:]

                # Process dataset
                if len(dataset.shape) == 1:
                    print(f'Skipping 1D dataset: {dataset_name}')
                    return

                if len(dataset.shape) == 2:
                    height, width = dataset.shape
                elif len(dataset.shape) == 3:
                    height, width = dataset.shape[1:3]
                else:
                    print(f'Unsupported dataset shape: {dataset.shape}')
                    return

                transform = from_origin(0, height, 1, 1)
                metadata = {
                    'driver': 'GTiff',
                    'count': 1,
                    'dtype': dataset.dtype,
                    'height': height,
                    'width': width,
                    'crs': 'EPSG:4326',
                    'transform': transform
                }

                with rasterio.open(output_geotiff_file, 'w', **metadata) as dst:
                    dst.write(dataset[0, :, :], 1)  # Use [0, :, :] to get the single band
            else:
                print(f'Dataset {dataset_name} not found in {input_hdf5_file}')
    except Exception as e:
        print(f'Error processing {dataset_name} from {input_hdf5_file}: {e}')


In [ ]:
import rasterio

def inspect_geotiff(geotiff_file):
    with rasterio.open(geotiff_file) as src:
        print(f"Driver: {src.driver}")
        print(f"Width: {src.width}")
        print(f"Height: {src.height}")
        print(f"Count: {src.count}")
        print(f"CRS: {src.crs}")
        print(f"Transform: {src.transform}")
        print(f"Dtype: {src.dtypes[0]}")

# Example usage
inspect_geotiff('/path/to/SIH2024OUTPUT.tif')


In [ ]:
import os

# Check if the file was created
def check_file_creation(file_path):
    if os.path.exists(file_path):
        print(f"File successfully created: {file_path}")
    else:
        print(f"File not found: {file_path}")

# Example usage
check_file_creation('/path/to/output.tif')


In [ ]:
import os

output_dir = '/path/to/output/directory'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [ ]:
output_geotiff_file = '/path/to/output/directory/output.tif'


In [ ]:
input_hdf5_file = '/content/drive/MyDrive/insat_data/SIH2024/3RIMG_04SEP2024_1015_L1B_STD_V01R00.h5'
dataset_name = 'your_dataset_name'  # Replace with actual dataset name
output_geotiff_file = '/path/to/output/directory/output.tif'

process_hdf5_to_geotiff(input_hdf5_file, output_geotiff_file, dataset_name)


In [ ]:
import h5py

with h5py.File(input_hdf5_file, 'r') as file:
    for key in file.keys():
        print(key)


In [ ]:
import os

input_hdf5_file = '/content/drive/MyDrive/insat_data/SIH2024/3RIMG_04SEP2024_1015_L1B_STD_V01R00.h5'

if not os.path.isfile(input_hdf5_file):
    print(f"File not found: {input_hdf5_file}")
else:
    print(f"File exists: {input_hdf5_file}")


In [ ]:
import h5py

input_hdf5_file = '/content/drive/MyDrive/insat_data/SIH2024/3RIMG_04SEP2024_1015_L1B_STD_V01R00.h5'

try:
    with h5py.File(input_hdf5_file, 'r') as file:
        print("HDF5 file opened successfully")
        # Print all available datasets in the file
        for key in file.keys():
            print(f"Dataset name: {key}")
except Exception as e:
    print(f"Error opening file: {e}")


In [ ]:
import os

output_directory = '/content/drive/MyDrive/insat_data/SIH2024/output_directory/'
output_geotiff_file = os.path.join(output_directory, 'output.tif')

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_hdf5_to_geotiff(input_hdf5_file, output_geotiff_file, dataset_name)


In [ ]:
import h5py
import rasterio
from rasterio.transform import from_origin

def process_hdf5_to_geotiff(input_hdf5_file, output_geotiff_file, dataset_name):
    try:
        with h5py.File(input_hdf5_file, 'r') as file:
            if dataset_name in file:
                dataset = file[dataset_name][:]

                # Process dataset
                if len(dataset.shape) == 1:
                    print(f'Skipping 1D dataset: {dataset_name}')
                    return

                if len(dataset.shape) == 2:
                    height, width = dataset.shape
                elif len(dataset.shape) == 3:
                    height, width = dataset.shape[1:3]
                else:
                    print(f'Unsupported dataset shape: {dataset.shape}')
                    return

                transform = from_origin(0, height, 1, 1)
                metadata = {
                    'driver': 'GTiff',
                    'count': 1,
                    'dtype': dataset.dtype,
                    'height': height,
                    'width': width,
                    'crs': 'EPSG:4326',
                    'transform': transform
                }

                with rasterio.open(output_geotiff_file, 'w', **metadata) as dst:
                    dst.write(dataset[0, :, :], 1)  # Use [0, :, :] to get the single band

                print(f'GeoTIFF created at {output_geotiff_file}')
            else:
                print(f'Dataset {dataset_name} not found in {input_hdf5_file}')
    except Exception as e:
        print(f'Error processing {dataset_name} from {input_hdf5_file}: {e}')

# Define paths and dataset name
input_hdf5_file = '/content/drive/MyDrive/insat_data/SIH2024/3RIMG_04SEP2024_1015_L1B_STD_V01R00.h5'
dataset_name = 'your_dataset_name'  # Replace with actual dataset name
output_directory = '/content/drive/MyDrive/insat_data/SIH2024/output_directory/'
output_geotiff_file = os.path.join(output_directory, 'output.tif')

# Ensure output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_hdf5_to_geotiff(input_hdf5_file, output_geotiff_file, dataset_name)


In [ ]:
import h5py

input_hdf5_file = '/content/drive/MyDrive/insat_data/SIH2024/3RIMG_04SEP2024_1015_L1B_STD_V01R00.h5'

try:
    with h5py.File(input_hdf5_file, 'r') as file:
        # List all datasets in the file
        def printname(name):
            print(name)

        file.visit(printname)  # This will print all dataset names
except Exception as e:
    print(f"Error opening file: {e}")


In [ ]:
import h5py
import rasterio
from rasterio.transform import from_origin
import os

def process_hdf5_to_geotiff(input_hdf5_file, output_geotiff_file, dataset_name):
    try:
        with h5py.File(input_hdf5_file, 'r') as file:
            if dataset_name in file:
                dataset = file[dataset_name][:]

                # Process dataset
                if len(dataset.shape) == 1:
                    print(f'Skipping 1D dataset: {dataset_name}')
                    return

                if len(dataset.shape) == 2:
                    height, width = dataset.shape
                elif len(dataset.shape) == 3:
                    height, width = dataset.shape[1:3]
                else:
                    print(f'Unsupported dataset shape: {dataset.shape}')
                    return

                transform = from_origin(0, height, 1, 1)
                metadata = {
                    'driver': 'GTiff',
                    'count': 1,
                    'dtype': dataset.dtype,
                    'height': height,
                    'width': width,
                    'crs': 'EPSG:4326',
                    'transform': transform
                }

                with rasterio.open(output_geotiff_file, 'w', **metadata) as dst:
                    if len(dataset.shape) == 3:  # Handle 3D datasets
                        dst.write(dataset[0, :, :], 1)  # Use the first band
                    else:
                        dst.write(dataset, 1)  # Write 2D dataset directly

                print(f'GeoTIFF created at {output_geotiff_file}')
            else:
                print(f'Dataset {dataset_name} not found in {input_hdf5_file}')
    except Exception as e:
        print(f'Error processing {dataset_name} from {input_hdf5_file}: {e}')

# Define paths and dataset name
input_hdf5_file = '/content/drive/MyDrive/insat_data/SIH2024/3RIMG_04SEP2024_1015_L1B_STD_V01R00.h5'
dataset_name = 'IMG_MIR'  # Replace with your chosen dataset name
output_directory = '/content/drive/MyDrive/insat_data/SIH2024/output_directory/'
output_geotiff_file = os.path.join(output_directory, 'output.tif')

# Ensure output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

process_hdf5_to_geotiff(input_hdf5_file, output_geotiff_file, dataset_name)


In [ ]:
def convert_all_datasets(input_hdf5_file, output_directory):
    try:
        with h5py.File(input_hdf5_file, 'r') as file:
            for dataset_name in file:
                output_geotiff_file = f'{output_directory}/{dataset_name}.tif'
                process_hdf5_to_geotiff(input_hdf5_file, output_geotiff_file, dataset_name)
                print(f'Converted {dataset_name} to GeoTIFF')
    except Exception as e:
        print(f'Error converting datasets: {e}')

# Example usage
convert_all_datasets('/content/drive/MyDrive/insat_data/SIH2024/3RIMG_04SEP2024_1015_L1B_STD_V01R00.h5', '/content/drive/MyDrive/insat_data/SIH2024/output_directory')


In [ ]:
output_geotiff_file = '/content/drive/MyDrive/insat_data/SIH2024/output.tif'


In [ ]:
!pip install rasterio
!pip install matplotlib


In [ ]:
import rasterio

def inspect_geotiff(file_path):
    with rasterio.open(file_path) as src:
        print("Metadata:")
        print(src.meta)
        print("\nBands:")
        for band in src.indexes:
            print(f"Band {band}:")
            print(f" - Data type: {src.dtypes[band-1]}")
            print(f" - Shape: {src.shape}")
            print(f" - CRS: {src.crs}")
            print(f" - Transform: {src.transform}")

# Replace with your GeoTIFF file path
geotiff_file_path = '/content/drive/MyDrive/insat_data/SIH2024/output_directory/output.tif'
inspect_geotiff(geotiff_file_path)


In [ ]:
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show

def plot_geotiff(file_path):
    with rasterio.open(file_path) as src:
        plt.figure(figsize=(10, 10))
        show(src, cmap='gray')
        plt.title('GeoTIFF Visualization')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.show()
geotiff_file_path='/content/drive/MyDrive/insat_data/SIH2024/output_directory/IMG_MIR.tif'
plot_geotiff(geotiff_file_path)


In [ ]:
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show

def plot_geotiff(file_path):
    with rasterio.open(file_path) as src:
        plt.figure(figsize=(10, 10))
        show(src, cmap='gray')  # You can change cmap based on data type
        plt.title('IMG_MIR GeoTIFF Visualization')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.show()

# Path to the GeoTIFF file
plot_geotiff(geotiff_file_path)


In [ ]:
def read_partial_geotiff(file_path, window):
    with rasterio.open(file_path) as src:
        data = src.read(1, window=window)  # Read data from the specified window
        transform = src.window_transform(window)  # Get the transform for the window
    return data, transform

# Example usage
from rasterio.windows import Window

# Define a window (e.g., xoff, yoff, width, height)
window = Window(100, 100, 256, 256)
data, transform = read_partial_geotiff(geotiff_file_path, window)

# Plot the partial data
plt.figure(figsize=(10, 10))
plt.imshow(data, cmap='gray')
plt.title('Partial View of IMG_MIR GeoTIFF')
plt.xlabel('Pixel X')
plt.ylabel('Pixel Y')
plt.show()


In [ ]:
def manipulate_geotiff(file_path, scale_factor=1.0, offset=0.0):
    with rasterio.open(file_path) as src:
        data = src.read(1)  # Read the entire band
        data = data * scale_factor + offset  # Apply scale and offset

    plt.figure(figsize=(10, 10))
    plt.imshow(data, cmap='gray')
    plt.title(f'Manipulated View of IMG_MIR GeoTIFF (Scale: {scale_factor}, Offset: {offset})')
    plt.xlabel('Pixel X')
    plt.ylabel('Pixel Y')
    plt.show()

# Example usage
manipulate_geotiff(geotiff_file_path, scale_factor=1.5, offset=10)


In [ ]:
!pip install rasterio numpy matplotlib pillow


In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Define file path
tiff_file_path = '/content/drive/MyDrive/insat_data/SIH2024/output_directory/IMG_MIR.tif'

# Open the GeoTIFF file
with rasterio.open(tiff_file_path) as src:
    # Read the data
    data = src.read(1)  # Read the first band
    # Get metadata
    metadata = src.meta
    # Get affine transform and CRS
    transform = src.transform
    crs = src.crs

# Display metadata
print(f'Metadata: {metadata}')
print(f'CRS: {crs}')


In [ ]:
# Display the GeoTIFF image
plt.imshow(data, cmap='gray')
plt.colorbar()
plt.title('GeoTIFF Image')
plt.show()


In [ ]:
# Apply a basic transformation (e.g., normalize)
normalized_data = (data - np.min(data)) / (np.max(data) - np.min(data))

# Save the manipulated image
output_path = '/content/drive/MyDrive/insat_data/SIH2024/output_directory/IMG_MIR_normalized.tif'
with rasterio.open(
    output_path, 'w', driver='GTiff', height=normalized_data.shape[0], width=normalized_data.shape[1],
    count=1, dtype='float32', crs=crs, transform=transform
) as dst:
    dst.write(normalized_data, 1)

# Display the manipulated image
plt.imshow(normalized_data, cmap='gray')
plt.colorbar()
plt.title('Normalized GeoTIFF Image')
plt.show()


In [ ]:
!pip install ipywidgets


In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from PIL import Image


In [ ]:
def load_geotiff(file_path):
    with rasterio.open(file_path) as src:
        data = src.read(1)  # Read the first band
        transform = src.transform
        crs = src.crs
    return data, transform, crs


In [ ]:
def band_arithmetic(file_path1, file_path2, operation='add'):
    data1, _, _ = load_geotiff(file_path1)
    data2, _, _ = load_geotiff(file_path2)

    if operation == 'add':
        result = data1 + data2
    elif operation == 'subtract':
        result = data1 - data2
    elif operation == 'multiply':
        result = data1 * data2
    elif operation == 'divide':
        result = np.divide(data1, data2, out=np.zeros_like(data1), where=data2!=0)
    else:
        raise ValueError("Operation not supported")

    return result


In [ ]:
def apply_color_adjustment(file_path, brightness=1.0, contrast=1.0):
    data, transform, crs = load_geotiff(file_path)
    adjusted_data = (data - np.mean(data)) * contrast + np.mean(data) * brightness
    adjusted_data = np.clip(adjusted_data, 0, 255)  # Ensure values are within valid range

    return adjusted_data


In [ ]:
def interactive_band_arithmetic(file_path1, file_path2):
    operation = widgets.Dropdown(
        options=['add', 'subtract', 'multiply', 'divide'],
        value='add',
        description='Operation:',
    )

    @interact(operation=operation)
    def display_band_arithmetic(operation):
        result = band_arithmetic(file_path1, file_path2, operation)
        plt.imshow(result, cmap='gray')
        plt.colorbar()
        plt.title(f'Band Arithmetic - {operation}')
        plt.show()

# Example usage:
file_path1 = '/content/drive/MyDrive/insat_data/SIH2024/output_directory/IMG_MIR.tif'
file_path2 = '/content/drive/MyDrive/insat_data/SIH2024/output_directory/IMG_TIR1.tif'
interactive_band_arithmetic(file_path1, file_path2)


In [ ]:
def interactive_color_adjustment(file_path):
    brightness = widgets.FloatSlider(
        value=1.0,
        min=0.0,
        max=2.0,
        step=0.1,
        description='Brightness:',
    )

    contrast = widgets.FloatSlider(
        value=1.0,
        min=0.0,
        max=2.0,
        step=0.1,
        description='Contrast:',
    )

    @interact(brightness=brightness, contrast=contrast)
    def display_color_adjustment(brightness, contrast):
        adjusted_data = apply_color_adjustment(file_path, brightness, contrast)
        plt.imshow(adjusted_data, cmap='gray')
        plt.colorbar()
        plt.title('Color Adjusted GeoTIFF Image')
        plt.show()

# Example usage:
file_path = '/content/drive/MyDrive/insat_data/SIH2024/output_directory/IMG_MIR.tif'
interactive_color_adjustment(file_path)


In [ ]:
!pip install streamlit pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show

def load_geotiff(file_path):
    with rasterio.open(file_path) as src:
        img = src.read(1)  # Read the first band
        meta = src.meta
    return img, meta

def show_image(image):
    fig, ax = plt.subplots()
    ax.imshow(image, cmap='gray')
    st.pyplot(fig)

def main():
    st.title("GeoTIFF Viewer and Manipulation")

    file_path = st.text_input("Enter the path to your GeoTIFF file", "/path/to/your/file.tif")

    if file_path:
        img, meta = load_geotiff(file_path)

        st.write("### Image Data")
        st.write(f"Shape: {img.shape}")
        st.write(f"Metadata: {meta}")

        st.write("### Image Preview")
        show_image(img)

        # Additional manipulations can be added here
        # e.g., adjusting brightness, applying filters, etc.

    else:
        st.write("Please enter a valid file path.")

if __name__ == "__main__":
    main()


In [ ]:
!pip install --upgrade pyngrok


In [ ]:
!streamlit run /content/app.py


In [ ]:


from pyngrok import ngrok


# Set your ngrok authtoken
ngrok.set_auth_token('YOUR KEY')

# Start a new ngrok tunnel
public_url = ngrok.connect(8501, "http")
print('Streamlit app is live at:', public_url)


In [ ]:
!apt-get install git


In [ ]:
!git config --global user.name "prashantxy"
!git config --global user.email "pdubey1924@gmail.com"


In [ ]:
!git clone https://github.com/prashantxy/INSAT.git
